In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np
import operator

In [2]:
alunos = pd.read_csv("../alunosUFCGAnon.csv")
pre_requisitos = pd.read_csv("../preRequisitos.csv")
periodo_disc = pd.read_csv("../periodoDisciplinas.csv")

In [3]:
# ajustando data frame que servirá de entrada
alunos_cc = alunos.query("Cod_Curso == 14102100 & Tipo == 'Obrigatória' ")
# alunos_cc.sort_values('Media_Disciplina', ascending=False).drop_duplicates(['Nome_Disciplina','Matricula'])
COMPUTACAOtdf = pd.pivot_table(alunos_cc, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
notas_alunos = alunos_cc
notas_alunos['Matricula'] = alunos_cc['Matricula'].map(lambda x: x.lstrip('B'))
notas_alunos = pd.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# cadeiras que não exigem prerrequisito
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                     'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                       'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']


/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
notas_aprovados = alunos_cc.loc[alunos_cc.Situacao == 'Aprovado']
notas_aprovados['Matricula'] = notas_aprovados['Matricula'].map(lambda x: x.lstrip('B'))
notas_aprovados = pd.pivot_table(notas_aprovados, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')

/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
%run util.ipynb
corr_disciplinas = notas_alunos.corr() 

In [40]:
# guardandos as regressoes para cada disciplina em um dicionario
disciplinas_regressoes = {}
regressoes = {}
regressao = linear_model.LinearRegression()
for disciplina in periodo_disc["DISCIPLINA"]:
    # pegando o periodo da disciplina da iteração
    periodo = periodo_disc.loc[periodo_disc["DISCIPLINA"] == disciplina]['PERIODO'].values[0]
    # pegando as disciplinas que são do mesmo periodo ou anteriores a disciplina em questão
    disciplinas_regressao = periodo_disc.loc[periodo_disc["PERIODO"]<=periodo]["DISCIPLINA"].values
    disciplinas_regressao = np.setdiff1d(corr_disciplinas.loc[disciplinas_regressao].sort_values(by=disciplina,ascending=False).drop([disciplina])[[disciplina]][:5].index, disciplina)
    
    disciplinas_regressoes[disciplina] = disciplinas_regressao
    disciplinas_regressao = notas_alunos[np.append(disciplinas_regressao,disciplina)].dropna(axis=0,how="any")

    X = disciplinas_regressao.loc[:,disciplinas_regressao.columns != disciplina]
    y = disciplinas_regressao.loc[:,disciplinas_regressao.columns == disciplina]
    regressoes[disciplina] = regressao.fit(X,y)



In [39]:
alunos_predicao = notas_aprovados.sample(n=5)
for index, aluno in alunos_predicao.iterrows():
    sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                        'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                         'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                           'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']
    
    # coletando quais cadeiras o aluno já foi aprovado
    cadeiras_aluno = aluno
    cadeiras_pagas = cadeiras_aluno.dropna().to_frame()

    cadeira_possivel = []

    # vendo quais cadeiras sem prerrequisito já foram pagas pelo aluno
    for cadeira in sem_prerequisito:
        if not cadeira in cadeiras_pagas.index:
            cadeira_possivel.append(cadeira)
    prox_possiveis_cadeiras = prox_cadeiras_nome(cadeiras_pagas, cadeira_possivel, pre_requisitos)
    
    dict_notas = {}
    for cadeira in prox_possiveis_cadeiras:
        if not (cadeiras_aluno[disciplinas_regressoes[cadeira]].isnull().values.any()):
            dict_notas[cadeira]=(regressoes[cadeira].predict(np.array(cadeiras_aluno[disciplinas_regressoes[cadeira]].fillna(0).values.reshape(1,-1))))
        
 
    
    
    # imprimindo notas preditas
    list_notas = sorted(dict_notas.items(), key = operator.itemgetter(1), reverse = True)
    print("Notas do historico do aluno:")
    print(aluno.T.dropna())
    print("-----------")
    print("Notas preditas: ")
    if not list_notas:
        print("Sem recomendaçao")
    else:
        for cadeira in list_notas: 
            print(cadeira[0], ": ", cadeira[1][0][0])
    
    print("================ \n")


Notas do historico do aluno:
Nome_Disciplina
ALGEBRA LINEAR I                          5.0
CALCULO DIFERENCIAL E INTEGRAL I          7.0
DIREITO E CIDADANIA                       7.0
ENGENHARIA DE SOFTWARE I                  7.3
FUNDAMENTOS DE FÍSICA CLÁSSICA            7.0
FUNDAMENTOS DE FÍSICA MODERNA             7.6
GERÊNCIA DA INFORMAÇÃO                    7.6
INFORMÁTICA E SOCIEDADE                   7.5
INTRODUÇÃO A COMPUTAÇÃO                   7.0
LABORATÓRIO DE PROGRAMAÇÃO I              5.4
LABORATÓRIO DE PROGRAMAÇÃO II             8.6
LEITURA E PRODUCAO DE TEXTOS              8.7
MATEMÁTICA DISCRETA                       6.1
METODOLOGIA CIENTÍFICA                    8.6
METODOS ESTATISTICOS                      7.7
PROBABILIDADE E ESTATISTICA               5.9
PROGRAMAÇÃO I                             7.1
PROGRAMAÇÃO II                            6.7
SISTEMAS DE INFORMAÇÃO I                  6.3
TEORIA DOS GRAFOS                         5.5
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍT